In [ ]:
!nvidia-smi

Mon Aug  2 23:24:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torch_optimizer
!pip install transformers
!pip install pytorchtools

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import json
import matplotlib.pyplot as plt
# from adamp import AdamP
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split


import torch
import torch_optimizer as optim
#from pytorch_lightning.callbacks.early_stopping import EarlyStopping
#import pytorch_lightning as pl

from transformers import ElectraTokenizer, ElectraForSequenceClassification,AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


# 그래프를 출력창에서 바로 볼 수 있게함
%matplotlib inline 


In [ ]:
my_learning_rate = 3E-6 # default is 5E-5
my_adam_epsilon = 1E-8 # default is 1E-8
my_number_of_epochs = 12
my_warmup = 3
my_mini_batch_size = 128
max_len = 32

In [ ]:
"""
GPU를 사용 가능한지 여부를 True, False로 나타냄
torch.cuda.is_available()
 
GPU 이름
torch.cuda.get_device_name(0)
"""
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('I will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

I will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
#################

#################
## Import DATA, submission file
PATH = '/content/drive/MyDrive/gh/dacon_newstopic'
train      = pd.read_csv(PATH + '/DATA/train_clean_data.csv')
test       = pd.read_csv(PATH + '/DATA/test_clean_data.csv')
submission = pd.read_csv(PATH + '/DATA/sample_submission.csv')
topic_dict = pd.read_csv(PATH + '/DATA/topic_dict.csv')

In [ ]:
# train data와 label 데이터 설정
texts = train['c_title']
labels = train['topic_idx']

In [ ]:
for n in range(200, 1000):
    print(texts[n], "(LABEL:", labels[n], ")")

In [ ]:
"""
Electra is selected here, its pretraining method is more advanced than BERT's MLM. 
AdamW is Adam with weight decay correction.
"""
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator") 
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v2-discriminator",num_labels=7) #let's try out electra's base discriminator

# tensor를 만든후 GPU에 할당하고 싶을 때  

# a = torch.tensor([1., 2.], device=cuda)
 
# b = torch.tensor([1., 2.]).cuda()

# b2 = torch.tensor([1., 2.]).to(device=cuda)


# 아니면 Model을 선언후 모든 parameter를 
# model.cuda()를 통해 GPU에 로딩할 수 있음

model.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
indices=tokenizer.batch_encode_plus(texts,
                                    max_length=max_len,
                                    add_special_tokens=True, 
                                    return_attention_mask=True,
                                    pad_to_max_length=True,
                                    truncation=True)

# bert와 달리(bert는 3개) input data가 2개임

input_ids=indices["input_ids"]
attention_masks=indices["attention_mask"]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:

"""
input_ids=indices["input_ids"]
attention_masks=indices["attention_mask"]
"""
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=42, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=42, test_size=0.2)

In [ ]:

# 만들어진 모든 data를 torch.tensor을 통해 tensor로 변환
# torch.long 은  np.int64 와 같음

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
validation_labels = torch.tensor(validation_labels.tolist(), dtype=torch.long) # list 변환 해줘야함
train_masks = torch.tensor(train_masks, dtype=torch.long)
validation_masks = torch.tensor(validation_masks, dtype=torch.long)

In [ ]:

# pytorch는  torch.utils.data.Dataset으로 Custom Dataset을 만들고, 
# torch.utils.data.DataLoader로 데이터를 불러옴 

batch_size = my_mini_batch_size

"""Create the DataLoader for our training set."""
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

#### DataLoader ###
# DataLoader를 통해 반복분을 돌리면  (batch_size, *(data.shape))의 형태의 Tensor로 데이터가 반환
# sampler : index를 컨트롤하는 방법, 즉 데이터의 index를 원하는 방식대로 조정
# sampler option 

# SequentialSampler : 항상 같은 순서
# RandomSampler : 랜덤, replacemetn 여부 선택 가능, 개수 선택 가능

# SubsetRandomSampler : 랜덤 리스트, 위와 두 조건 불가능
# WeigthRandomSampler : 가중치에 따른 확률
# BatchSampler : batch단위로 sampling 가능
# DistributedSampler : 분산처리 (torch.nn.parallel.DistributedDataParallel과 함께 사용)


""" Create the DataLoader for our validation set."""
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = my_learning_rate, #args.learning_rate
                  eps = my_adam_epsilon  #args.adam_epsilon

                )

"""
my_learning_rate = 3E-6 # default is 5E-5
my_adam_epsilon = 1E-8 # default is 1E-6
my_number_of_epochs = 7
my_warmup = 3
my_mini_batch_size = 128

"""


# epoch 수 설정
epochs = my_number_of_epochs

# Total number of training steps is number of batches * number of epochs.
# batchs * epoch 수 만큼의 훈련을 함
total_steps = len(train_dataloader) * epochs

""" warmup
내가 정한 learning rate가 0.01이라고 한다면 처음 10 step 동안은 
0.001, 0.002, 0.003 ~ 0.01까지 선형적으로 조금씩만 증가하는 learning rate을 사용

이러한 방식은 warmup step을 따로 정해줘야 함
"""

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = my_warmup, 
                                            num_training_steps = total_steps)

In [ ]:

#about evalution mert - here we use accuracy, which is good enough because the data is
#is binary classified and distribution is pretty even between positive and negative
#however better evaluation should be use later F1 or AUC ROC because emergency are
#events rare tweets

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random

# Base on GLUE from huggingface, this is classification problem best suit our problem, here, look at the
# example Python code from hungingface gtihub, here is the here training loop

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128



# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progress update every 100 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # `batch` contains three pytorch tensors: [0]: input ids ,[1]: attention masks,[2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear any previously calculated gradients.
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Evaluate the model on this training batch.
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        loss = outputs[0]

        # Accumulate the training loss over all of the batches 
        total_loss += loss.item()
        loss.backward()

        # Clip the norm of the gradients to 1.0. to prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)       
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
   

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
      
print("")
print("Training complete!")




======== Epoch 1 / 12 ========
Training...
  Batch    50  of    286.    Elapsed: 0:00:19.
  Batch   100  of    286.    Elapsed: 0:00:37.
  Batch   150  of    286.    Elapsed: 0:00:56.
  Batch   200  of    286.    Elapsed: 0:01:15.
  Batch   250  of    286.    Elapsed: 0:01:34.

  Average training loss: 1.76
  Training epoch took: 0:01:47

======== Epoch 2 / 12 ========
Training...
  Batch    50  of    286.    Elapsed: 0:00:19.
  Batch   100  of    286.    Elapsed: 0:00:37.
  Batch   150  of    286.    Elapsed: 0:00:56.
  Batch   200  of    286.    Elapsed: 0:01:15.
  Batch   250  of    286.    Elapsed: 0:01:34.

  Average training loss: 1.09
  Training epoch took: 0:01:47

======== Epoch 3 / 12 ========
Training...
  Batch    50  of    286.    Elapsed: 0:00:19.
  Batch   100  of    286.    Elapsed: 0:00:37.
  Batch   150  of    286.    Elapsed: 0:00:56.
  Batch   200  of    286.    Elapsed: 0:01:15.
  Batch   250  of    286.    Elapsed: 0:01:33.

  Average training loss: 0.71
  Traini

In [ ]:
# Validation
print("")
print("Running Validation...")

t0 = time.time()
model.eval()

preds=[]
true=[]

# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Evaluate data for one epoch
for batch in validation_dataloader:
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():        

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # Get the "logits" output by the model. The "logits" are the output values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    preds.append(logits)
    true.append(label_ids)
    # Calculate the accuracy for this batch.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy

    # Track the number of batches
    nb_eval_steps += 1
valid_flat_predictions = [item for sublist in preds for item in sublist]
# Report the final accuracy for this validation run.
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))
    


Running Validation...
  Accuracy: 0.87
  Validation took: 0:00:09


In [ ]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in preds for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
valid_flat_predictions = np.argmax(valid_flat_predictions, axis=1).flatten()
# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true for item in sublist]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(flat_predictions,flat_true_labels))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83      1102
           1       0.81      0.84      0.82      1183
           2       0.73      0.79      0.76      1365
           3       0.87      0.87      0.87      1117
           4       0.93      0.90      0.92      1541
           5       0.97      0.96      0.96      1466
           6       0.90      0.91      0.90      1357

    accuracy                           0.87      9131
   macro avg       0.87      0.87      0.87      9131
weighted avg       0.87      0.87      0.87      9131



In [ ]:
text1 = test['c_test']

indices1=tokenizer.batch_encode_plus(text1,
                                     max_length=32,
                                     add_special_tokens=True, 
                                     return_attention_mask=True,
                                     pad_to_max_length=True,
                                     truncation=True)
input_ids1=indices1["input_ids"]
attention_masks1=indices1["attention_mask"]

prediction_inputs1= torch.tensor(input_ids1)
prediction_masks1 = torch.tensor(attention_masks1)


# Set the batch size.  
batch_size = my_mini_batch_size

# Create the DataLoader.
prediction_data1 = TensorDataset(prediction_inputs1, prediction_masks1)
prediction_sampler1 = SequentialSampler(prediction_data1)
prediction_dataloader1 = DataLoader(prediction_data1, sampler=prediction_sampler1, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs1)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions = []

# Predict 
for batch in prediction_dataloader1:

  # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
    b_input_ids1, b_input_mask1 = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs1 = model(b_input_ids1, token_type_ids=None, 
                      attention_mask=b_input_mask1)

    logits1 = outputs1[0]

  # Move logits and labels to CPU
    logits1 = logits1.detach().cpu().numpy()
  
  
  # Store predictions and true labels
    predictions.append(logits1)

flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions_1 = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

Predicting labels for 9,131 test sentences...


In [ ]:
import torch.nn.functional as F

In [ ]:
z = torch.FloatTensor(flat_predictions_1)
probs_1 = F.softmax(z, dim=1)
probs_1 = (probs_1).tolist()
flat_predictions

array([0, 3, 2, ..., 2, 0, 2])

In [ ]:
i, j, k=  [], [], []
m, n, o = [], [], []
l = []
for row in probs_1:
  i.append(row[0])
  j.append(row[1])
  k.append(row[2])
  l.append(row[3])
  m.append(row[4])
  n.append(row[5])
  o.append(row[6])


In [ ]:
submission_1 = submission
submission_1['0'] = i
submission_1['1'] = j
submission_1['2'] = k
submission_1['3'] = l
submission_1['4'] = m
submission_1['5'] = n
submission_1['6'] = o

In [ ]:
submission_1.topic_idx = flat_predictions

In [ ]:
submission_1[:12]

,index,topic_idx,0,1,2,3,4,5,6
0,45654,0,0.799334,0.023457,0.112502,0.043212,0.009871,0.008006,0.003617
1,45655,3,0.005623,0.004369,0.011433,0.968275,0.003912,0.004565,0.001822
2,45656,2,0.116257,0.049765,0.804231,0.008282,0.007168,0.004123,0.010173
3,45657,2,0.022933,0.010146,0.912220,0.029071,0.006535,0.004358,0.014736
4,45658,3,0.005306,0.004042,0.014073,0.966153,0.003800,0.004663,0.001963
5,45659,0,0.837380,0.028925,0.074656,0.043698,0.006368,0.006166,0.002809
6,45660,5,0.003512,0.002115,0.002885,0.002479,0.008621,0.975553,0.004835
7,45661,3,0.006418,0.004316,0.019176,0.959936,0.003636,0.004392,0.002125
8,45662,4,0.003747,0.004506,0.005790,0.003162,0.969520,0.003757,0.009517
9,45663,4,0.013780,0.071222,0.015737,0.007525,0.872717,0.008308,0.010712


In [ ]:
# pred = np.argmax(predictions, axis = 1)
submission = pd.read_csv("/content/drive/MyDrive/gh/dacon_newstopic/DATA/sample_submission.csv",error_bad_lines=False)
submission.topic_idx = flat_predictions
submission.sample(3)

#submission.to_csv(path, index = False)

,index,topic_idx
4043,49697,6
3573,49227,6
4505,50159,3


In [ ]:
submission

,index,topic_idx
0,45654,0
1,45655,3
2,45656,2
3,45657,2
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,2
9129,54783,0


In [ ]:
crosstab = pd.crosstab(validation_labels, valid_flat_predictions, rownames=['real'], colnames=['pred'])
crosstab

pred,0,1,2,3,4,5,6
real,,,,,,,
0,866,36,54,14,14,8,3
1,116,993,74,6,27,9,5
2,62,115,1082,86,39,13,86
3,32,12,67,977,22,5,7
4,17,16,12,19,1386,19,19
5,7,3,7,12,8,1406,8
6,2,8,69,3,45,6,1229


In [ ]:
submission.to_csv("/content/drive/MyDrive/gh/dacon_newstopic/DATA/koelectra_dh3.csv",index = False)
# submission.to_csv('bert_baseline_1.csv',index = True)

In [ ]:
!pip install /content/drive/MyDrive/gh/dacon_newstopic/DATA/dacon_submit_api-0.0.4-py3-none-any.whl

Processing ./drive/MyDrive/gh/dacon_newstopic/DATA/dacon_submit_api-0.0.4-py3-none-any.whl


In [ ]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'/content/drive/MyDrive/gh/dacon_newstopic/DATA/koelectra_dh3.csv', # 파일경로
'9df81cd09375bf4a64d8e0e74e73a5bf83190442474256e9da79999b0cd10c32',  # 개인토큰
'235747', # 대회 id
'학습기계',  # 팀이름
'koelectra_dh3') # 노트

{'isSubmitted': True, 'detail': 'Success'}
